# **📄 Document type classification baseline code**

**Contents**
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


# 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [1]:
%%time
# 필요한 라이브러리를 설치합니다. 1분걸림
%pip install timm

CPU times: user 55.8 ms, sys: 13.7 ms, total: 69.5 ms
Wall time: 8.28 s


## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [2]:
### 전역변수 설정
home = '/content/drive/MyDrive/AILAB/CV/Contest/'


import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

import cv2
from PIL import Image

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.13 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
c:\Users\kkukk\AppData\Local\Programs\Python\Python312\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#import gc
#gc.collect()
#torch.cuda.empty_cache()

In [5]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [6]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [7]:
# 학습을 위한 함수입니다.
def train_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [3]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# data config
data_path = '/home/data'

# model config
model_name = 'efficientnet_b5' # 'resnet50' 'efficientnet-b0', ...

# 하이퍼파라미터를 설정합니다.
img_size = 242  #224
#의미: 입력 이미지의 크기를 지정합니다. 모델에 입력하기 전에 이미지를 이 크기로 조정합니다.
#변화:
#크기를 늘리면 모델이 더 많은 세부 정보를 학습할 수 있지만, 계산 비용이 증가합니다.
#크기를 줄이면 계산 비용은 감소하지만, 정보 손실로 인해 성능이 떨어질 수 있습니다.
LR = 1e-4
#의미: 학습률을 지정합니다. 모델 파라미터를 업데이트할 때 사용되는 비율입니다.
#변화:
#학습률이 높으면 학습이 빠르게 진행되지만, 최적값을 놓치거나 불안정할 수 있습니다.
#학습률이 낮으면 안정적으로 학습되지만, 시간이 오래 걸릴 수 있습니다.
EPOCHS = 5
#의미: 전체 데이터셋을 학습할 횟수를 지정합니다.
#변화:
#에포크 수가 많으면 모델이 데이터에 대해 더 잘 학습할 수 있지만, 과적합(overfitting)의 위험이 있습니다.
#에포크 수가 적으면 과소적합(underfitting)이 발생할 수 있습니다.
BATCH_SIZE = 2
#의미: 한 번에 학습할 데이터 샘플의 수를 지정합니다.
#변화:
#배치 크기가 크면 학습이 안정적이고, GPU의 효율성을 높일 수 있습니다. 하지만 메모리 사용량이 증가합니다.
#배치 크기가 작으면 메모리 사용량이 적고, 미세한 업데이트가 가능하지만, 학습이 불안정할 수 있습니다.
num_workers = 2
#의미: 데이터 로딩을 위해 사용할 병렬 작업자의 수를 지정합니다.
#변화:
#작업자 수가 많으면 데이터 로딩 속도가 빨라져 학습이 더 원활하게 진행될 수 있습니다.
#너무 많은 작업자를 사용하면 시스템의 다른 작업과 충돌하거나 메모리 문제가 발생할 수 있습니다.

cpu


In [6]:
pip install gputil

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7402 sha256=0dcaa2901c6d1260f4d98a7d21c67cf177f68fe70d77b9be36182ecdc4bf1f43
  Stored in directory: c:\users\kkukk\appdata\local\pip\cache\wheels\92\a8\b7\d8a067c31a74de9ca252bbe53dea5f896faabd25d55f541037
Successfully built gputil
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import GPUtil

# 사용 가능한 모든 GPU 정보 가져오기
gpus = GPUtil.getGPUs()

for gpu in gpus:
    print(f"GPU ID: {gpu.id}")
    print(f"GPU Name: {gpu.name}")

GPU ID: 0
GPU Name: NVIDIA GeForce RTX 2080 Ti


In [13]:
import torch

# PyTorch 버전 확인
print(f"PyTorch version: {torch.__version__}")

# CUDA 사용 가능 여부 확인
print(f"Is CUDA available: {torch.cuda.is_available()}")

# CUDA 버전 확인
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

# cuDNN 버전 확인
print(f"cuDNN version: {torch.backends.cudnn.version()}")

# GPU 이름 확인
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.4.0+cpu
Is CUDA available: False
cuDNN version: None


In [12]:
import torch
torch.cuda.get_device_name(0)
torch.cuda.is_available()
torch.__version__

AssertionError: Torch not compiled with CUDA enabled

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [9]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [10]:
 os.chdir('/content/drive/MyDrive/AILAB/CV/Contest')

In [11]:
os.getcwd()

'/content/drive/MyDrive/AILAB/CV/Contest'

In [12]:
# 디렉토리 내 파일 개수 확인
def count_files_in_directory(directory):
    # 디렉토리 내 파일 목록 가져오기
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    # 파일 개수 반환
    return len(files)

# Dataset 정의
trn_dataset = ImageDataset(
    csv = 'csv/train_3_step1.csv',
    path = 'data/train_2_45D' ,
    transform=trn_transform
)
tst_dataset = ImageDataset(
    csv = 'csv/sample_submission.csv',
    path = 'data/test',
    transform=tst_transform
)

print("train dataset 개수:", len(trn_dataset), "  test dataset 개수:", len(tst_dataset))

#file_count1 = count_files_in_directory('data/train_2_45D')  # 수정된 변수 사용
#file_count2 = count_files_in_directory('data/test')  # 수정된 변수 사용
#print(f"train 파일 개수: {file_count1}  test 파일 개수: {file_count2}")

train dataset 개수: 48000   test dataset 개수: 3140


In [13]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [14]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [15]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=8
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [16]:
print(f"model: {model_name}  |  img_size: {img_size}  |  Learning Rate: {LR}  |  EPOCHS: {EPOCHS}  |  BATCH_SIZE: {BATCH_SIZE}  |  num_workers: {num_workers}")

model: efficientnet_b5  |  img_size: 242  |  Learning Rate: 0.0001  |  EPOCHS: 5  |  BATCH_SIZE: 2  |  num_workers: 2


In [21]:
%%time
#브레이크
### 모델을 저장
# 메모리 가능 확인 기다리는 시간 30초

#from torch.utils.tensorboard import SummaryWriter

# TensorBoard SummaryWriter
#writer = SummaryWriter(log_dir='home/runs/exp1')

model_checkpoint = "45D" #input()
save_dir = home + 'model/model_' + model_checkpoint
os.makedirs(save_dir, exist_ok=True)

for epoch in range(EPOCHS):
    ret = train_epoch(trn_loader, model, optimizer, loss_fn, device=device)  # train_epoch 함수를 호출하여 한 에포크 동안 모델을 학습시키고 결과를 반환받습니다.
    ret['epoch'] = epoch  # 반환된 결과 딕셔너리에 현재 에포크 번호를 추가합니다.

    log = ""  # 로그 메시지를 저장할 문자열 변수를 초기화합니다.
    for k, v in ret.items():  # 결과 딕셔너리의 각 항목을 순회하면서 키와 값을 로그 문자열에 추가합니다.
        log += f"{k}: {v:.4f}\n"  # 키와 값을 포맷팅하여 로그 문자열에 추가합니다.
    print(log)  # 완성된 로그 메시지를 출력합니다.

    # 모델의 상태를 저장합니다.
    checkpoint_path = os.path.join(save_dir, f"model_epoch_{epoch}.pth")
    torch.save(model.state_dict(), checkpoint_path)

    # TensorBoard에 기록합니다.
    #writer.add_scalar('Loss/train', ret['train_loss'], epoch)
    #writer.add_scalar('Accuracy/train', ret['train_acc'], epoch)
    #writer.add_scalar('F1_Score/train', ret['train_f1'], epoch)

#writer.close()"""

  0%|          | 0/24000 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [18]:
#torch.cuda.empty_cache()

In [19]:
#!nvidia-smi

In [20]:
교차검증 구현 브레이크

SyntaxError: invalid syntax (<ipython-input-20-ab45c6d23a3c>, line 1)

In [ ]:
### K-Fold 교차 검증 구현

def k_fold_cross_validation(k, dataset, model, optimizer, loss_fn, device, epochs):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        print(f"Fold {fold + 1}")

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=16, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_subset, batch_size=16, shuffle=False, num_workers=4)

        for epoch in range(epochs):
            train_ret = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
            val_ret = validate(val_loader, model, loss_fn, device)

            print(f"Epoch {epoch + 1}/{epochs} - "
                  f"Train Loss: {train_ret['train_loss']:.4f} - Train Acc: {train_ret['train_acc']:.4f} - Train F1: {train_ret['train_f1']:.4f} - "
                  f"Val Loss: {val_ret['val_loss']:.4f} - Val Acc: {val_ret['val_acc']:.4f} - Val F1: {val_ret['val_f1']:.4f}")

        results.append(val_ret)

    return results


In [ ]:
### K-fold 모델학습

# K-Fold 교차 검증 실행
k = 5  # K 값 설정
epochs = 10  # 에포크 수 설정
results = k_fold_cross_validation(k, dataset, model, optimizer, loss_fn, device, epochs)

# 최종 결과 출력
for fold, result in enumerate(results):
    print(f"Fold {fold + 1} - "
          f"Val Loss: {result['val_loss']:.4f} - Val Acc: {result['val_acc']:.4f} - Val F1: {result['val_f1']:.4f}")


# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [ ]:
브레이크

In [ ]:
### 저장된 모델 불러오기

# 에포크 4에서 저장된 모델 파일 경로
epoch_to_load = 4
checkpoint_path = os.path.join(save_dir, f"model_epoch_{epoch_to_load}.pth")

# 저장된 모델 상태를 불러옵니다
model.load_state_dict(torch.load(checkpoint_path))


In [ ]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

In [ ]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [ ]:
sample_submission_df = pd.read_csv("/home/data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [ ]:
pred_df.to_csv("/home/data/test_step1.csv", index=False)

In [ ]:
# pred_df Macro F1 score 계산
#from sklearn.metrics import f1_score
#f1_score(sample_submission_df['target'], pred_df['target'], average='macro')

In [ ]:
pred_df.head(5)

In [ ]:
pred_df['target'].value_counts()

In [ ]:
#ex = pred_df.reset_index()

ex1 = pred_df.loc[pred_df['target'] == 0]
ex2 = pred_df.loc[pred_df['target'] != 0]

In [ ]:
print(ex1, ex2)

In [ ]:
ex1.to_csv('/home/step1.csv')

In [ ]:
# 1부터 17까지의 숫자로 폴더 생성
for i in range(0, 17):
    folder_name = str(i)
    folder_path = os.path.join('/home/data/cl/', folder_name)
    os.makedirs(folder_path, exist_ok=True)
    #print(f"폴더 생성: {folder_path}")

class_df = pred_df

for i in range(0, len(class_df)) :
    print(class_df['ID'][i], class_df['target'][i] )
    img = cv2.imread('/home/data/test2x/' + class_df['ID'][i])
    cv2.imwrite('/home/data/cl/' + str(class_df['target'][i]) + '/' + class_df['ID'][i], img)

